In [ ]:
!pip install pandas
!pip install requests
!pip install beautifulsoup4

#爬數量

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 
import re

In [ ]:
def post(YM, total):
  url = 'https://www.sitca.org.tw/ROC/Industry/IN2629.aspx'
  with requests.Session() as s:
    page = s.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
 
    payload_loginPage = {
        "ctl00$ContentPlaceHolder1$ddlQ_YM" : YM,
        "ctl00$ContentPlaceHolder1$rdo1" : "rbClass",
        "ctl00$ContentPlaceHolder1$ddlQ_Class" : "AA1",
        "ctl00$ContentPlaceHolder1$BtnQuery" : "查詢"
    }
 
    payload_loginPage["__VIEWSTATE"] = soup.select_one("#__VIEWSTATE")["value"]
    payload_loginPage["__VIEWSTATEGENERATOR"] = soup.select_one("#__VIEWSTATEGENERATOR")["value"]
    payload_loginPage["__EVENTVALIDATION"] = soup.select_one("#__EVENTVALIDATION")["value"]
 
  headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'}
  
  ss = s.post(url, data=payload_loginPage, headers = headers)
  soup = BeautifulSoup(ss.content, 'lxml')
  DTeven = soup.select(".DTeven")
  DTodd = soup.select(".DTodd")

  
  #data1
  DT_m1=[]
  for DT in DTeven:
    match_result = re.match(r'<td align="left" class="DTeven">(.*)</td>', str(DT), re.I)
    if match_result!=None:
      m = match_result.group(1)
      if m:
        DT_m1.append(m.encode('raw_unicode_escape').decode())
  DTSum1=[DT_m1[dtm] for dtm in range(len(DT_m1)) if dtm%3==2 and DT_m1[dtm-2]=="國內上市"]
  
  #data2
  DT_m2=[]
  for DT in DTodd:
    match_result = re.match(r'<td align="left" class="DTodd">(.*)</td>', str(DT), re.I)
    if match_result!=None:
      m = match_result.group(1)
      if m:
        DT_m2.append(m.encode('raw_unicode_escape').decode())
  DTSum2=[DT_m2[dtm] for dtm in range(len(DT_m2)) if dtm%3==2 and DT_m2[dtm-2]=="國內上市"]
 
  #合併
  DTSum = DTSum1+DTSum2
  #小計
  print(DTSum)
  for sum in DTSum:
    if sum in total:
      total[sum] += 1
    else:
      total[sum] = 1
  #print(total)

In [ ]:
#執行畫面
total={} 
YM="202207"
post(YM, total)
 
data=[]
for n,v in sorted(total.items(),key=lambda x:x[1],reverse=True):
  sv=[]
  sv.append(n)
  sv.append(v)
  data.append(sv)
df = pd.DataFrame(data,columns=['投資標的','數量總計'])
 
df.to_excel(YM+'.xlsx')

['漢翔', '南電', '台積電', '開發金', '美時製藥', '上緯投控', '元晶', '中信金', '漢翔', '漢翔', '中磊', '高力', '智易', '上緯投控', '台積電', '中磊', '智易', '高力', '昇陽半導體', '台積電', '漢翔', '高力', '華通', '台積電', '中信金', '統一', '中鋼', '安集', '台積電', '欣興', '南電', '台積電', '金像電', '欣興', '健策', '聯發科', '美時製藥', '臺企銀', '裕融', '台積電', '中華電', '金像電', '群光', '欣興', '台積電', '長榮航', '聯發科', '台達電', '兆豐金', '台積電', '長榮航', '貿聯-KY', '聯發科', '日月光投控', '南亞', '中租-KY', '長榮', '統一', '華航', '台積電', '嘉澤', '台達電', '聯發科', '同欣電', '台積電', '中華電', '華通', '中磊', '智邦', '台積電', '欣興', '昇陽半導體', '美時製藥', '台積電', '中磊', '創意', '華通', '昇陽半導體', '台積電', '日月光投控', '廣達', '力成', '臻鼎-KY', '中華電', '中砂', '聯發科', '臻鼎-KY', '台積電', '金像電', '臻鼎-KY', '華通', '貿聯-KY', '台積電', '創意', '欣興', '台達電', '鴻海', '金像電', '欣興', '智易', '台積電', '台塑', '聯電', '欣興', '川湖', '台積電', '聯發科', '鴻海', '川湖', '日月光投控', '台積電', '台塑', '川湖', '南亞科', '台積電', '奇鋐', '嘉澤', '中磊', '振樺電', '健策', '嘉澤', '台積電', '長榮航', '晶碩', '新唐', '奇鋐', '台積電', '嘉澤', '穎崴', '臻鼎-KY', '中磊', '台積電', '智伸科', '同欣電', '祥碩', '台塑', '台塑化', '長榮航', '中租-KY', '上海商銀', '台積電', '鴻海', '奇鋐', '川湖', '昇陽半導體', '貿聯-KY', '南電', '景

In [ ]:
df

,投資標的,數量總計
0,台積電,115
1,欣興,73
2,嘉澤,47
3,金像電,44
4,台達電,43
...,...,...
134,佳必琪,1
135,健和興,1
136,慧洋-KY,1
137,矽力-KY,1


#爬全部資料

In [ ]:
from operator import index
def post2(YM):
  url = 'https://www.sitca.org.tw/ROC/Industry/IN2629.aspx'
  with requests.Session() as s:
      page = s.get(url)
      soup = BeautifulSoup(page.content, 'lxml')
  
      payload_loginPage = {
          "ctl00$ContentPlaceHolder1$ddlQ_YM" : YM,
          "ctl00$ContentPlaceHolder1$rdo1" : "rbClass",
          "ctl00$ContentPlaceHolder1$ddlQ_Class" : "AA1",
          "ctl00$ContentPlaceHolder1$BtnQuery" : "查詢"
      }
  
      payload_loginPage["__VIEWSTATE"] = soup.select_one("#__VIEWSTATE")["value"]
      payload_loginPage["__VIEWSTATEGENERATOR"] = soup.select_one("#__VIEWSTATEGENERATOR")["value"]
      payload_loginPage["__EVENTVALIDATION"] = soup.select_one("#__EVENTVALIDATION")["value"]
  
  headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'}
    
  ss = s.post(url, data=payload_loginPage, headers = headers)
  soup = BeautifulSoup(ss.content, 'lxml')
  DTeven = soup.select(".DTeven")
  DTodd = soup.select(".DTodd")
  #data1
  name=[]
  DT_m1=[]
  DT1=[]
  i=1
  for DT in DTeven:
      match_result = re.match(r'<td align="[a-z]*" class="DTeven">(.*)</td>', str(DT), re.I)
      if match_result!=None:
        m1 = match_result.group(1)
        if m1:
          DT_m1.append(m1.encode('raw_unicode_escape').decode())
          i+=1
          if i>7:
            DT1.append(DT_m1) 
            i=1
            DT_m1=[]

      match = re.match(r'<td align="left" class="DTeven" rowspan="10">(.*)</td>', str(DT), re.I)
      if match!=None:
        m2 = match.group(1)
        if m2:
          for k in range(10):
            name.append(m2.encode('raw_unicode_escape').decode())
  #data2
  DT_m2=[]
  DT2=[]
  j=1
  for DT in DTodd:
      match_result = re.match(r'<td align="[a-z]*" class="DTodd">(.*)</td>', str(DT), re.I)
      if match_result!=None:
        m = match_result.group(1)
        if m:
          DT_m2.append(m.encode('raw_unicode_escape').decode())
          j+=1
          if j>7:
            DT2.append(DT_m2) 
            j=1
            DT_m2=[] 
  #合併
  DTSum = []
  for i in range(len(DT1)):
    DTSum.append(DT1[i])
    DTSum.append(DT2[i])

  df = pd.DataFrame(data=DTSum,columns=['名次','標的種類','標的代號','標的名稱','金額','受益權單位數','占基金淨資產價值之比例(%)'])
  df['基金名稱'] = name
  df['年'] = YM[0:4]
  df['月'] = YM[4:6]
  df2 = df[['年','月','基金名稱','標的代號','標的名稱','金額','占基金淨資產價值之比例(%)']]
  df2['年'] = df2['年'].astype('int')
  df2['月'] = df2['月'].astype('int')
  df2['基金名稱'] = df2['基金名稱'].astype('string')
  df2['標的代號'] = df2['標的代號'].astype('string')
  df2['標的名稱'] = df2['標的名稱'].astype('string')
  df2['金額'] = df2['金額'].str.replace(',','').astype('int')
  df2['占基金淨資產價值之比例(%)'] = df2['占基金淨資產價值之比例(%)'].astype('float')
  #df2.set_index('年', inplace=True)
  df2.to_csv('基金.csv',encoding='utf-8-sig') 
  return df2

#股票名稱加代號

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

res = requests.get("https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y")
    
df1 = pd.read_html(res.text)[0]
df1 = df1.drop(index=[0])     
        
res = requests.get("https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=4&industry_code=&Page=1&chklike=Y")
df2 = pd.read_html(res.text)[0]
df2 = df2.drop(index=[0])

df_name = list(df1[3])+list(df2[3])
df_val = list(df1[2])+list(df2[2])
df = pd.DataFrame(df_val,columns=['標的代號'])
df['標的名稱'] = df_name
df.to_csv('標的.csv',encoding='utf-8-sig') 
df

In [ ]:
for d in range(len(df['標的代號'])):
  print(f"INSERT INTO 標的 VALUES ({df['標的代號'][d]},N'{df['標的名稱'][d]}');")

INSERT INTO 標的 VALUES (1101,N'台泥');
INSERT INTO 標的 VALUES (1102,N'亞泥');
INSERT INTO 標的 VALUES (1103,N'嘉泥');
INSERT INTO 標的 VALUES (1104,N'環泥');
INSERT INTO 標的 VALUES (1108,N'幸福');
INSERT INTO 標的 VALUES (1109,N'信大');
INSERT INTO 標的 VALUES (1110,N'東泥');
INSERT INTO 標的 VALUES (1201,N'味全');
INSERT INTO 標的 VALUES (1203,N'味王');
INSERT INTO 標的 VALUES (1210,N'大成');
INSERT INTO 標的 VALUES (1213,N'大飲');
INSERT INTO 標的 VALUES (1215,N'卜蜂');
INSERT INTO 標的 VALUES (1216,N'統一');
INSERT INTO 標的 VALUES (1217,N'愛之味');
INSERT INTO 標的 VALUES (1218,N'泰山');
INSERT INTO 標的 VALUES (1219,N'福壽');
INSERT INTO 標的 VALUES (1220,N'台榮');
INSERT INTO 標的 VALUES (1225,N'福懋油');
INSERT INTO 標的 VALUES (1227,N'佳格');
INSERT INTO 標的 VALUES (1229,N'聯華');
INSERT INTO 標的 VALUES (1231,N'聯華食');
INSERT INTO 標的 VALUES (1232,N'大統益');
INSERT INTO 標的 VALUES (1233,N'天仁');
INSERT INTO 標的 VALUES (1234,N'黑松');
INSERT INTO 標的 VALUES (1235,N'興泰');
INSERT INTO 標的 VALUES (1236,N'宏亞');
INSERT INTO 標的 VALUES (1256,N'鮮活果汁-KY');
INSERT INTO 標的 VALU